# Security+ - Firewall Setup

#### Pre-requisites:
    * Basic network understanding
        * Ports
    * Wireshark
    * Virtualbox w/ Windows
    * Virtualbox w/ Ubuntu
    
##### Desired Outcome(s):
    
    * Understand the basic purpose of a host based firewall
    * Understand the terms `egress` and `ingress` traffic
    * Understand how to set firewall default policies
    * Understand the various types of firewalls
    * Understand how to configure a Windows firewall
    * Understand how to configure Linux `ufw` and high level understanding of `iptables`
    * Understand various ways to allow/deny traffic by:
        * Application
        * Port
        * IP Address/CIDR Range
        * Protocol
    * Understand how to test and troubleshoot firewall rules
    * Understand that firewalls are considered "protective" controls
    
#### What is a firewall?
   * A firewall is considered a **protective** control as oppoed to a **detective** control like an IDS
   * A firewall is generally hardware or software that filters what traffic is allowed into a network or a network interface
   * A properly configured firewall should block ALL traffic by default and whitelist what kind of traffic is allowed explicitly
   

#### What are the various types of firewall?
    
    * Stateless/packet filtering firewall (iptables)
        * Layer 2 usually
        * Filters by fields in the packet headers
        * Easily spoofable by manipulating packet headers
    * Stateful firewall
        * More intelligent, knows things like if a 3 way handshake was performed before allowing HTTP traffic
        * Layer 3/4 to get context
    * Application layer firewall
        * Layer 5 and above
        * Analyzes payload contents for things accepted by the application
    * Next-gen firewall
        * All of the above w/ additional features like IPS nnd WAF
        
#### Firewall Topologies

Learn the following types of firewall topologies in the Security+ material:

    * Dual-Homed Host Firewalls
        * Two network interfaces, one for public network, one for private
        * Packets make it to the network interface
        * DEMO: Create a Digital Ocean droplet with private networking
    * Screened-Host Firewalls
        * Bastion host
        * A physical router/firewall in front of the host
        * Packets never make it to the host
    * Screened-Subnet Firewalls aka DMZ
        * Additional router in front of the firewall to filter by destination before deeper inspection by screened-host firewall
        * Only traffic is accepted between the screened-host/router
        
[Firewall Topologies and Diagrams](http://www.boran.com/security/it12-firewall.html#Heading53)
        
#### Nmap port status definitions

    * open
    * filtered
    * closed
    * unfiltered


#### VirtualBox NAT Network Setup

* Open VirtualBox

* Select the Wirehark->Preferences option

![Wireshark Preferences](img/wireshark-preferences.png)

* Select the Network tab

![Wireshark Network Tab](img/wireshark-network-tab.png)


* Click the Add Network icon, right-click on the newly created `NatNetwork` and verify that you have the following settings:

![Wireshark NAT Network Settings](img/wireshark-nat-configuration.png)

* Click OK to save your changes and exit the VirtualBox Preferences dialogue box

* Select the Network tab on **ALL** your virtual machines with the following settings:

    * Kali
    * Ubuntu 18.04
    * Windows 10


* Select the `NAT Network` option in the `Attached to:` drop down menu


* Select the `NatNetwork` option


* Click the Advanced option


* Select the `Allow All` option in the `Promiscuous Mode:` drop down menu 


![title](img/virtualbox-nat-network-setup.png)


* Start Kali3 virtual machine 

* Start Ubuntu virtual machine

* Start Windows virtual machine


#### Setting up a Linux firewall

* Inspect currently installed firewall utilities and default settings

* Run `sudo iptables -L or sudo iptables --list`
    
    * Go over INPUT(ingress), OUTPUT(egress), FORWARD
    * What are reasons a server would need to send packets as OUTPUT or FORWARD
    
* Run `sudo iptables -F` if any rules exist

* Run `sudo iptables -S or sudo iptables --list-rules`
    * Less detail, just shows rules, no verbose output

* Run `sudo ufw status` (optional)

* Scan w/ nmap from Kali with `nmap -sT 10.0.2.8`

* Check status of firewall, setup default strict rules
    * Block incoming, outgoing, forward by default
    
        * `sudo iptables -P INPUT DROP`
        * `sudo iptables -P OUTPUT DROP`
        * `sudo iptables -P FORWARD DROP`
    
* Scan w/ nmap from Kali with `nmap -sT 10.0.2.8`


* BEST PRACTICE: Allow loopback connections

```
sudo iptables -A INPUT -i lo -j ACCEPT
sudo iptables -A OUTPUT -o lo -j ACCEPT
```

* Go to the Windows box and try to get to https://internalportal.com

* Use `conntrack` module to allow previously established connections for both incoming and outgoing:

    * `sudo iptables -A INPUT -m conntrack --ctstate ESTABLISHED,RELATED -j ACCEPT`
    * `sudo iptables -A OUTPUT -m conntrack --ctstate ESTABLISHED -j ACCEPT`


* Open port 80, 443, 53 to all traffic
    * `sudo iptables -A INPUT -p tcp --dport 443 -j ACCEPT`
    * `sudo iptables -A INPUT -p tcp --dport 80 -j ACCEPT`
    
* Scan from Kali again, ports accessible

* Flush or delete rules and set stricter rules for only access from Windows

    * `sudo iptables -A INPUT -p tcp --dport 80 -m conntrack --ctstate NEW,ESTABLISHED -j ACCEPT`
    * `sudo iptables -A OUTPUT -p tcp --sport 80 -m conntrack --ctstate ESTABLISHED -j ACCEPT`
    * Repeat for HTTPS traffic
    

* Go to the Windows box and try to get to https://internalportal.com
    * Shouldn't be able to get here based on DNS being blocked
    
* Harden rules that only traffic from Windows can talk to DNS
    * `sudo iptables -A INPUT -p TCP -s 10.0.0.6 --source-port 1024:65535 -d 10.0.0.8 --destination-port 53 -j ACCEPT`
    
* Harden rules that only traffic from Windows can talk to HTTP and HTTPS
    * `sudo iptables -A INPUT -p TCP -s 10.0.0.6 --source-port 1024:65535 -d 10.0.0.8 --destination-port 80 -j ACCEPT`
    * `sudo iptables -A INPUT -p TCP -s 10.0.0.6 --source-port 1024:65535 -d 10.0.0.8 --destination-port 443 -j ACCEPT`
    
    
* Block a specific IP address

    * `sudo iptables -A INPUT -s 15.15.15.51 -j DROP`

* Get past firewall w/ custom packets on Kali
    * `nmap -f 10.0.2.8` (FAIL)
    * `nmap -mtu 8 10.0.2.8` (FAIL)
    * `nmap -sT -PN -spoof-mac 0800275283B9 10.0.2.8` (FAIL)
    * `nmap -e eth0 -S 10.0.2.6 10.0.2.8` (PASS)
    
* To view traffic activity per rule use: `sudo iptables -L INPUT -v`
        * Should see traffic under the DROP rule as well as any ACCEPT traffic from Windows host
        
* Show ordering of rules and precedence

* To show line numbers use: `sudo iptables -L --line-numbers`

* Insert rule into chain with: `sudo iptables -I INPUT 4 new_rule_here`

* To delete a specific rule use: `sudo iptables -D INPUT 3`

* Save rules with the following steps

```
# install iptables-persistent
sudo apt-get install iptables-persistent

# save current firewall rules
sudo netfilter-persistent save

```

* EXERCISE: Setup FTP and DNS access


#### Setup firewall logging
After all of your defined rules you will want to add custom logging behavior, this **MUST** be at the end of your log definitions

Reference for fields here: https://www.thegeekstuff.com/2012/08/iptables-log-packets/

```
iptables -N LOGGING
iptables -A INPUT -j LOGGING
iptables -A OUTPUT -j LOGGING
iptables -A LOGGING -m limit --limit 2/min -j LOG --log-prefix "IPTables-Dropped: " --log-level 4
iptables -A LOGGING -j DROP`
```

To view if logs work, run the following command: `sudo tail -F -n 100 /var/log/kern.log`

To change the default location to it's own log create the following file `/etc/rsyslog.d/my_iptables.conf` with the following commands:

```
sudo vim /etc/rsyslog.d/my_iptables.conf
```

Add the following configuration for regex matching of our log-prefix from above
```
:msg,contains,"IPTables-Dropped: " /var/log/iptables.log
```

Restart the service and test with Nmap from Kali while viewing the log file
```
sudo systemctl restart rsyslog
sudo tail -F -n 100 /var/log/iptables.log
```

#### Nmap firewall evasion techniques

    * Decoys (-D) - Mask the source of scan, makes it hard to block
        * `nmap -sT --top-ports 10 -D 10.0.2.6 10.0.2.8`
        * `nmap -sS -D Rnd:5 10.0.2.8`

    * Ping suppression (-Pn)
        * `nmap -sT -Pn 10.0.2.8`

    * TCP ACK Scan/Half-open scan (-sA) - Send only replies, useful for stateless firewall and IDS that are setup to allow replies to SYN requests
        * `nmap -sA 10.0.2.8`
        
    * TCP Window Scan (-sW) - Very similar to the ACK scan with a little difference, the TCP window scan has been designed to differentiate between open and closed ports instead of showing unfiltered
        * `nmap -sW 10.0.2.8`
        
    * Fragmentation (-f) - Break the header down into smaller fragmented packets, pieces are reassembled at the destination. 
        * `nmap -sT -f 10.0.2.8`
        * `nmap -sT -ff 10.0.2.8` (further break down headers)
        
    * Changing data length (--data-length n) Can also get around signatures w/ larger than normal packets. Adds data length to end of packet. `-mtu` packets must be in multiples of 8
        * `nmap -sT --data-length 120 10.0.2.8`
        * `nmap -sT -mtu 8 10.0.2.8`
        * `nmap -sT -mtu 24 10.0.2.8`
        
    * Mac Address Spoofing (--spoof-mac) - Used to hide the actual source of traffic or to spoof as another host on the same network
        * `nmap --spoof-mac Cisco 10.0.2.8`
        
    * Timing Delay (--scan_delay 5ms) - Used to slow down requests to not set off alarms
        * `nmap -sS --scan_delay 5s 10.0.2.8`
        
    * Bad checksums (--badsum) - Improperly configured IDS or firewalls 
        * `nmap -sS --badsum 10.0.2.8`
        
    * Random host order (--randomize-hosts) - Used to not set off a firewall, best used with timing delay
        * `nmap --randomize-hosts 10.0.2.6-100`
        
    * Idle scan (-sI) - only works with servers that respond with linear fragment id's. Used to completely mask the true target of the scan, but never receive the actual response. Cannot decipher between closed and filtered ports with this scan method. 
        * `nmap -sI 10.0.2.7 10.0.2.8`
        * https://nmap.org/book/idlescan.html
    
    * Using multiple combinations to avoid detection
        * `nmap -sS -T1 -f --badsum --data-length 1200 --proxies sock4://176.57.216.214:80,http://213.85.92.10:80 44.35.33.21`


#### Resources
- [Nmap Usage Mind Map](https://securityonline.info/nmap-usage-mindmap-network-administrators/)
- [Install iptables on Ubuntu](https://www.digitalocean.com/community/tutorials/how-to-set-up-a-firewall-using-iptables-on-ubuntu-14-04)
- [List and delete iptables on Ubuntu](https://www.digitalocean.com/community/tutorials/how-to-list-and-delete-iptables-firewall-rules)
- [Common iptables rules and commands](https://www.digitalocean.com/community/tutorials/iptables-essentials-common-firewall-rules-and-commands)
- [Linux Iptables block common attacks](https://www.cyberciti.biz/tips/linux-iptables-10-how-to-block-common-attack.html)
- [Nmap Bypass Firewalls and IDS (Original)](https://nmap.org/book/man-bypass-firewalls-ids.html)
- [Nmap Bypass Firewalls and IDS (2018 Edition)](https://resources.infosecinstitute.com/nmap-evade-firewall-scripting/)
- [IDS, IPS AND FIREWALL EVASION USING NMAP](https://technologyredefine.blogspot.com/2017/09/ids-ips-and-firewall-evasion-using-nmap.html?m=1)
- [iptables logging setup](https://www.thegeekstuff.com/2012/08/iptables-log-packets/)
- [Testing firewall rules w/ hping3](http://www.draconyx.net/articles/firewall-testing-using-hping3.html)